In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,24343
2,Huelva,Confirmados PDIA 14 días,1086
3,Huelva,Tasa PDIA 14 días,"211,62577703295204"
4,Huelva,Confirmados PDIA 7 días,380
5,Huelva,Total Confirmados,24573
6,Huelva,Curados,14097
7,Huelva,Fallecidos,270


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  24343.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6502.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1219 personas en los últimos 7 días 

Un positivo PCR cada 416 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,24343.0,380.0,1086.0,513170.0,74.049535,211.625777,108.0
Huelva-Costa,14846.0,241.0,709.0,289548.0,83.233177,244.864409,65.0
Condado-Campiña,6740.0,123.0,306.0,156231.0,78.729574,195.863817,39.0
Huelva (capital),6502.0,118.0,346.0,143837.0,82.037306,240.550067,33.0
Cartaya,915.0,38.0,66.0,20083.0,189.214759,328.636160,10.0
Bollullos Par del Condado,591.0,23.0,45.0,14387.0,159.866546,312.782373,8.0
Moguer,803.0,15.0,48.0,21867.0,68.596515,219.508849,8.0
Almonte,826.0,37.0,84.0,24507.0,150.977272,342.759212,7.0
Ayamonte,1180.0,13.0,45.0,21104.0,61.599697,213.229719,6.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villablanca,168.0,4.0,14.0,2885.0,138.648180,485.268631,4.0
Escacena del Campo,136.0,6.0,8.0,2287.0,262.352427,349.803236,2.0
Almonte,826.0,37.0,84.0,24507.0,150.977272,342.759212,7.0
Lepe,1964.0,25.0,94.0,27880.0,89.670014,337.159254,3.0
Cartaya,915.0,38.0,66.0,20083.0,189.214759,328.636160,10.0
Villanueva de los Castillejos,146.0,4.0,9.0,2825.0,141.592920,318.584071,1.0
Bollullos Par del Condado,591.0,23.0,45.0,14387.0,159.866546,312.782373,8.0
Isla Cristina,1981.0,17.0,66.0,21393.0,79.465246,308.512130,2.0
Punta Umbría,650.0,13.0,38.0,15355.0,84.662976,247.476392,4.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Nerva,152.0,1.0,7.0,5169.0,19.346102,135.422712,1.0,0.142857
Aracena,456.0,1.0,6.0,8255.0,12.113870,72.683222,0.0,0.166667
Sierra de Huelva-Andévalo Central,2327.0,8.0,47.0,67391.0,11.871021,69.742250,3.0,0.170213
Palma del Condado (La),960.0,3.0,16.0,10801.0,27.775206,148.134432,-1.0,0.187500
Valverde del Camino,423.0,3.0,15.0,12750.0,23.529412,117.647059,-1.0,0.200000
Bonares,179.0,2.0,9.0,6060.0,33.003300,148.514851,2.0,0.222222
Palos de la Frontera,440.0,6.0,27.0,11742.0,51.098620,229.943792,5.0,0.222222
Cerro de Andévalo (El),135.0,1.0,4.0,2327.0,42.973786,171.895144,0.0,0.250000
Isla Cristina,1981.0,17.0,66.0,21393.0,79.465246,308.512130,2.0,0.257576
